In [1]:
import pandas as pd
import numpy as np
import re
import string
import os.path
from fuzzywuzzy import fuzz
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
import spacy
from spacy.matcher import PhraseMatcher
from spacy.pipeline import EntityRecognizer
from spacy.training import Corpus
from spacy import displacy
from spacy.tokens import Doc
from thinc.api import Config
from spacy.language import Language
from bs4 import BeautifulSoup
import requests
import urllib.parse
from lxml import etree

In [2]:
omit_in_search_str = {"UNITED STATES OF AMERICA", "MISC GENERAL MERCHANDISE", "VIDEO AMUSEMENT GAME SUPPLY"}

In [3]:
api_url = 'https://app.scrapingbee.com/api/v1/?'
api_key_old1 = 'PFAX0VZTO4EPTYIHED58F8GZ5E298GQWMETO6X6BO51Y43QQV80H1EFIP6J6EN7NKLJSBG5L6T2TZRU4'
api_key_old2 = 'ZST8Y9KSLMSRUANUZL7D31PW7ND24HPNPBUAPSU1BEXV835ZXUISY2D929FI8UQ55KA7MCUFM1GRCT75'
api_key = 'GG64U7ORER3EJ76EP7UQ5HS5TGKYA8YDTF2PAELH5N96LXDI49QPT0I1BQJ39U96ALIW1SFCU2KBQLIU'
ADDRESS = 0
BIZ_NAME = 1
UNWANTED_PHRASE = 2
TEST_DATA_PATH = "C:\Smrithi\Internships\Algotrak\algotrak-automation\TestDataSmrithi.xlsx"
TEST_DATA_PATH_REL = ".\..\datasets\TestDataSmrithi.xlsx"

cardinal = '(\d+[:]CARDINAL[;])'
fac = '(\d+[:]FAC[;])'
gpe = '(\d+[:]GPE[;])'
loc = '(\d+[:]LOC[;])'
ordinal = '(\d+[:]ORDINAL[;])'
org = '(\d+[:]ORG[;])'
product = '(\d+[:]PRODUCT[;])'
quantity = '(\d+[:]QUANTITY[;])'
work_of_art = '(\d+[:]WORK_OF_ART[;])'
date = '(\d+[:]DATE[;])'
punct = '(\d+[:]PUNCT[;])'
noent = '(\d+[:]NOENT[;])'
par_open = '('
par_close = ')'
or_pipe = '|'
star = '*'
plus = '+'
quest_mark = '?'
dollar = '$'

In [4]:
addr_pattern_pt1 = cardinal + star + fac + star + punct + star + loc + star + punct + star + ordinal + star
addr_pattern_pt2 = punct + star + par_open + fac + star + punct + star + par_close + star + gpe + plus + punct + star
addr_pattern_pt3 = par_open + gpe + star + punct + star + par_close + star + cardinal + star + punct + star + gpe + star
addr_pattern = addr_pattern_pt1 + addr_pattern_pt2 + addr_pattern_pt3
biz_name_pattern = par_open + product + or_pipe + work_of_art + par_close + star + org + plus
unwanted_phrase_pattern = par_open + product + or_pipe + work_of_art + or_pipe + noent + or_pipe + punct + par_close + plus + dollar

In [5]:
cols_req = ['Init', 'Gold Name', 'Gold Address', 'Match Type']
test_data = pd.read_excel(TEST_DATA_PATH_REL, usecols=cols_req)
test_data

,Init,Gold Name,Gold Address,Match Type
0,HUA FU FAT BULL SEAFOO RIVERSIDE CA 92507 UNIT...,92507,"RIVERSIDE, CA 92507, USA",2000
1,"KARZU INVESTMENT, LLC PHARR TX 78577 UNITED ST...",PHARR,"PHARR, TX 78577, USA",2000
2,14 RBRT GAHANNA COLUMBUS OH 43230 UNITED STATE...,43230,"GAHANNA, OH 43230, USA",2000
3,176 COMP KITCH 877-280-73 TROY MI 48084 UNITED...,48084,"TROY, MI 48084, USA",2000
4,4TH STREET PARKING BERKELEY CA 94710 UNITED ST...,"4TH STREET PARKING, LLC","BERKELEY, CA 94710, USA",2000
...,...,...,...,...
95,USAVEND AT AIR SERV SUGAR HILL GA 30518 UNITED...,30518,"SUGAR HILL, GA 30518, USA",2000
96,USAVEND AT COCA-COLA DALLAS NC 28034 UNITED ST...,DALLAS,"DALLAS, NC, USA",2000
97,VENDING BRANDON ROBB DES MOINES IA 50310 UNITE...,50310,"DES MOINES, IA 50310, USA",2000
98,VENDING CT VENDING CASTLE PINES CO 80108 UNITE...,CASTLE PINES,"CASTLE PINES, CO 80108, USA",2000


In [6]:
state_names = pd.read_csv("../datasets/state-name-codes.csv")
state_names.drop('Abbrev', axis=1, inplace=True)
state_names_dict = {}
for index, row in state_names.iterrows():
    state_names_dict[row["Code"]] = row["State"]
print(state_names_dict['GU'])

Guam


In [12]:
cust_ner = spacy.blank("en")
ner = cust_ner.add_pipe("ner")
train_corpus = Corpus("../datasets/train.spacy")
train_data = list(train_corpus(cust_ner))
cust_ner.initialize()
cust_ner.update(train_data)

# ------------------ OR -----------------------------

#config = Config().from_disk("./config.cfg")
#cust_ner = Language.from_config(config)


{'ner': 60528.98046875}

In [13]:
cust_ner = spacy.load("./output/model-best")
doc = cust_ner("GAMESTOP.COM 19 2700 MIA DAYTON OH 45459 UNITED STATES OF AMERICA VIDEO AMUSEMENT GAME SUPPLY")
for token in doc:
    print(token.text + "-" + token.ent_iob_ + "-" + token.ent_type_)

GAMESTOP.COM-O-
19-O-
2700-O-
MIA-O-
DAYTON-O-
OH-O-
45459-O-
UNITED-O-
STATES-O-
OF-O-
AMERICA-B-location
VIDEO-O-
AMUSEMENT-O-
GAME-O-
SUPPLY-O-


In [14]:
print(cust_ner.config.to_str())
cust_ner.config.to_disk("./config.cfg")

[paths]
train = "../datasets/train.spacy"
dev = "../datasets/dev.spacy"
vectors = null
init_tok2vec = null

[system]
seed = 0
gpu_allocator = null

[nlp]
lang = "en"
pipeline = ["ner"]
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
batch_size = 1000
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.HashEmbedCNN.v2"
pretrained_vectors = null
width = 96
depth = 4
embed_size = 2000
window_size = 1
maxout_pieces = 3
subword_features = true

[corpora]

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
gold_preproc = false
max_length = 0
limit = 0
augmenter = null

[corpora.train]
@r

In [15]:
def checkGoldOutput(nlp, test_data):
    test_data.replace(np.nan, '', regex=True, inplace=True) # in-place modification of test_data
    init_inputs_series = test_data.loc[:, 'Init'] # Values of Init column alone, as pandas series
    init_inputs_list = init_inputs_series.to_list() # init column values series converted to list
    init_input_docs = list(nlp.pipe(init_inputs_list)) # list of init column values as docs
    matcher = PhraseMatcher(nlp.vocab) # spacy phrase matcher instance

    # I will be using the substring 'united states of america'...
    # ...to deduce where the business name ends and address starts
    matcher.add("USA", [nlp("UNITED STATES OF AMERICA")])
    
    # Correct names and correct addresses are basically business names and addresses...
    # ...deduced from the init strings. I will be storing them in separate lists
    correct_names = []
    correct_addrs = []
    
    for doc in init_input_docs: # iterating thru init string docs
        matches = matcher(doc, as_spans=True)
        if len(matches) > 0: # because if this condition fails, a lotta errors are thrown
            
            # taking only the first match, in the event of there being...
            # ...more than one match
            req_match = matches[0]
            
            # flag to break out of the following while loop
            extracted = 0

            # offset value to calculate pos of token starting from the matched...
            # ...phrase 'united states of america'
            i = 1

            # this will just be a list of tokens of the address...
            # ...substring from init string
            inp_addr = []
            
            # while loop to extract tokens which should contain address data
            # the order of tokens extracted will be reversed since we are...
            # ...moving backwards from a particular point in the string
            while extracted == 0:
                token = doc[req_match.start - i]
                # the present conditions for considering a token as that...
                # ...having address info
                if token.is_alpha or token.is_digit or token.is_space or token.is_ascii:
                    inp_addr.append(token.text)
                    i += 1

                # city names like "new york" might not make it to the extracted tokens list
                # only 'york' will be extracted as the number of tokens extracted...
                # ...thus far is also considered
                # only way to check is by using info from the iob tagging in the data
                if i >= 4 and token.ent_iob_ in ['B', 'O']:
                    # we have got what we wanted - the extracted address tokens
                    extracted = 1
            
            # putting the extracted addr tokens in the correct order
            inp_addr.reverse()
            
            # the addr tokens list does not include the 'united states of america'...
            # ...phrase which was used to split the string. We thus add...
            # ...it to the list, but abbreviated
            inp_addr.append('USA')

            # so far we have the address substring info. We now get the tokens...
            # ...from the start of the string till the last token we extracted
            correct_names.append(doc[0: req_match.start - i + 1].text)
            
            # this is the string which will hold the complete address substring...
            # ...constructed using the 'join' function, delimited by space
            inp_addr_str = " "
            correct_addrs.append(inp_addr_str.join(inp_addr))

        # there must've been no matches, so we just append a "---"...
        # ...to each of the extracted-details lists
        else:
            correct_names.append("---")
            correct_addrs.append("---")
    
    # converting lists to pandas series
    correct_name_series = pd.Series(correct_names, name="Extracted Name")
    correct_addr_series = pd.Series(correct_addrs, name="Extracted Address")

    # getting gold outputs for names and addresses
    gold_outputs = test_data.loc[:, ['Gold Name', 'Gold Address']]
    
    # final results
    yes_or_no_list = []
    name_comp_list = []
    addr_comp_list = []
    
    # comparing each gold name and addr with extracted (correct) name and addr
    # to-do: rewrite this part as a separate function and use pd.series.apply() to perform the ops
    for i in range(gold_outputs.shape[0]):
        gold_name = str(gold_outputs.at[i, 'Gold Name'])
        corr_name = str(correct_name_series.iloc[i])
        gold_addr = str(gold_outputs.at[i, 'Gold Address'])
        corr_addr = str(correct_addr_series.iloc[i])
        name_score = fuzz.token_sort_ratio(gold_name, corr_name)
        addr_score = fuzz.token_sort_ratio(gold_addr, corr_addr)
        if name_score > 65 and addr_score > 50:
            yes_or_no_list.append('Y')
        else:
            yes_or_no_list.append('N')
        name_comp_list.append(name_score)
        addr_comp_list.append(addr_score)
            
    yes_or_no = pd.Series(yes_or_no_list, name="Y/N") 
    name_comp = pd.Series(name_comp_list, name="Name Comparison") 
    addr_comp = pd.Series(addr_comp_list, name="Address Comparison")
    col_names = ["Y/N", "Extracted Name", "Extracted Address", "Name Comparison", "Address Comparison"]
    return yes_or_no, correct_name_series, correct_addr_series, name_comp, addr_comp # return pandas series


In [16]:
def send_request(use_params):
    response = requests.get(
        url=api_url,
        params={
            'api_key': api_key,
            'url': use_params['url'],
            'custom_google': use_params['custom_google'],
        },        
    )
    print('Response HTTP Status Code: ', response.status_code)
    return(response)

In [17]:
def modifyString(in_strs):
    ret_strs = []
    for in_str in in_strs:
        ret_str = in_str
        for (k, v) in state_names_dict.items():
            pattern = "(?<![a-zA-Z])" + re.escape(k) + "(?![a-zA-Z])"
            ret_str = re.sub(pattern, v, ret_str, flags=re.I)
        ret_strs.append(ret_str)
    return ret_strs

stringg = "MMS MS MS MS907 MSS TX,PA,TN GGGmekgnejbiq"
string2 = "AAFES GUAM NEW MAIN STOR DALLAS TX 75265 UNITED STATES OF AMERICA MISC GENERAL MERCHANDISE"
string3 = "GAMESTOP.COM 19 2700 MIA DAYTON OH 45459 UNITED STATES OF AMERICA VIDEO AMUSEMENT GAME SUPPLY"
print(modifyString([string3]))

['GAMESTOP.COM 19 2700 MIA DAYTON Ohio 45459 UNITED STATES OF AMERICA VIDEO AMUSEMENT GAME SUPPLY']


In [18]:
def handleSpecialCases(token):
    entity_name = "NOENT"
    token_text = token.text
    if len(token_text) == 1 and token_text in string.punctuation:
        entity_name = 'PUNCT'
    return entity_name

In [19]:
def stringInfo(docs):
    for doc in docs:
        print(doc.text + "\n--------------------------------------------------------------------")
        for token in doc:
            print(str(token.i))
            print(token.text + ": " + token.ent_iob_ + "-" + token.ent_type_ + " ", end="")
            if spacy.explain(token.ent_type_) is not None:
                print(spacy.explain(token.ent_type_))
            else:
                print('No entity')
        print()

In [20]:
def removeUnwantedPhrases(proc_texts):
    ret_proc_texts = proc_texts
    for to_omit in omit_in_search_str:
        for i in range(len(ret_proc_texts)):
            new_str = re.sub(to_omit, "", ret_proc_texts[i], flags=re.I)
            ret_proc_texts[i] = new_str
    ret_proc_texts = [ret_proc_texts[i].strip(string.whitespace + string.punctuation) for i in range(len(ret_proc_texts))]
    return ret_proc_texts
textx= 'AAFES GUAM NEW MAIN STOR DALLAS TX 75265 UNITED STATES OF AMERICA MISC GENERAL MERCHANDISE'
removeUnwantedPhrases([textx])

['AAFES GUAM NEW MAIN STOR DALLAS TX 75265']

In [21]:
def textPreproc(nlp, texts, dont_remove_unwanted=0):
    preproc_texts = modifyString(texts)
    if dont_remove_unwanted == 0:
        proc_texts = removeUnwantedPhrases(preproc_texts)
        preproc_texts = proc_texts
    docs = nlp.pipe(proc_texts)
    return docs

In [22]:
def entityString(docs):
    ret_strs = []
    for doc in docs:
        proc_string = ""
        for token in doc:
            entity_name = ""
            if len(str(token.ent_type_)) == 0:
                entity_name = handleSpecialCases(token)
            else:
                entity_name = token.ent_type_
            add_str = str(token.i) + ":" + entity_name + ";"
            # print("Adding: " + add_str)
            proc_string += add_str
        ret_strs.append(proc_string)
    return ret_strs # return list of strings

In [23]:
def couldItBeAddr(match):
    ents = re.findall('[A-Za-z]+', match.group(0))
    ents_dict = {}
    ents_dict = dict((ents[i], i)  for i in range(len(ents)) if ents[i] not in ents_dict)
    ents_set = set(ents_dict.keys())
    if len(ents_set) < 2:
        return 0
    elif len(ents_set) == 2:
        if {"CARDINAL", "GPE"} == ents_set and ents_dict['CARDINAL'] < ents_dict['GPE']:
            return 0
        elif "PUNCT" in ents_set:
            return 0
    else:
        return 1

In [24]:
def couldItBeBizName(match):
    return 1

In [25]:
def couldItBeUnwantedPhrases(match):
    return 1

In [26]:
def checkValidity(match, flag):
    if flag == ADDRESS:
        return couldItBeAddr(match)
    elif flag == BIZ_NAME:
        return couldItBeBizName(match)
    elif flag == UNWANTED_PHRASE:
        return couldItBeUnwantedPhrases(match)

In [27]:
def extractPattern(pattern, ent_strs, flag=ADDRESS):
    full_index_list = []
    for ent_str in ent_strs:
        index_list = [] # corresponds to one set of matches
        matches = re.finditer(pattern, ent_str)
        for match in matches:
            if checkValidity(match, flag) == 0:
                continue
            else:
                ind = re.findall('\d+', match.group(0))
                indices = list(map(lambda x: int(x), ind))
                first_index = indices[0]
                last_index = indices[len(indices) - 1] + 1
                index_list.append([first_index, last_index]) # [first_index, last_index] corresponds to one match
        full_index_list.append(index_list)
    return full_index_list # return list of lists of index-pair lists

In [28]:
def findUnwantedPhrases(docs):
    ents_strs = entityString(docs)
    match_indices = extractPattern(unwanted_phrase_pattern, ents_strs, UNWANTED_PHRASE)
    match_strings_dict = getMatches(match_indices)
    changes_req = bool(match_strings_dict)
    if changes_req:
        for (k, v) in match_strings_dict:
            for match_str in v:
                omit_in_search_str.add(match_str.upper())
    return changes_req

In [29]:
def getMatches(search_results, orig_strings): # search_results is a list of lists containing match index-pairs
    i = 0
    matches_acr_src_res = {}
    for search_result in search_results:
        search_query = orig_strings[i]
        matches_in_src_res = []
        #print("Matches for text: " + search_query.text)
        for text_span in search_result:
            final_result = search_query[text_span[0]: text_span[1]].text.strip(string.whitespace + string.punctuation)
            #print(final_result)
            matches_in_src_res.append(final_result)
        matches_acr_src_res[i] = matches_in_src_res
        #print("------------------------------------------------------------------\n")
        i += 1
    return matches_acr_src_res # return dict with integer keys and list-of-matches values

In [30]:
print(spacy.util.get_installed_models())

nlp = spacy.load("en_core_web_trf")
nlp.resume_training()
#print(nlp.config.to_str())
print(nlp.pipe_names)

['en_core_web_trf']
['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [31]:
ret_tuple = checkGoldOutput(nlp, test_data) # y_or_n will hold pandas series
if os.path.exists('./output/testdata.xlsx'):
    test_data = pd.read_excel('./output/testdata.xlsx')

for ret_series in ret_tuple:
    if ret_series.name in test_data.columns:
        index_loc = test_data.columns.get_loc(ret_series.name)
        test_data.drop(ret_series.name, axis=1, inplace=True)
        test_data.insert(index_loc, ret_series.name, ret_series)
    else:
        test_data.insert(test_data.shape[1], ret_series.name, ret_series)

In [32]:
test_data.to_excel('./output/testdata1.xlsx', index=False)

In [33]:
for index, row in test_data.iterrows():
    doc = nlp(row['Init'])
    for token in doc:
        print(token.text + "-" + token.ent_iob_ + "-" + token.ent_type_)
    print("-------------------------------------------------------")

HUA-O-
FU-O-
FAT-O-
BULL-O-
SEAFOO-O-
RIVERSIDE-O-
CA-O-
92507-B-CARDINAL
UNITED-B-GPE
STATES-I-GPE
OF-I-GPE
AMERICA-I-GPE
RESTAURANTS-O-
-------------------------------------------------------
KARZU-B-ORG
INVESTMENT-I-ORG
,-I-ORG
LLC-I-ORG
PHARR-B-GPE
TX-B-GPE
78577-B-CARDINAL
UNITED-B-GPE
STATES-I-GPE
OF-I-GPE
AMERICA-I-GPE
RESTAURANTS-O-
-------------------------------------------------------
14-O-
RBRT-O-
GAHANNA-O-
COLUMBUS-B-GPE
OH-B-GPE
43230-B-CARDINAL
UNITED-B-GPE
STATES-I-GPE
OF-I-GPE
AMERICA-I-GPE
RESTAURANTS-O-
-------------------------------------------------------
176-O-
COMP-O-
KITCH-O-
877-O-
--O-
280-O-
--O-
73-O-
TROY-O-
MI-O-
48084-O-
UNITED-B-GPE
STATES-I-GPE
OF-I-GPE
AMERICA-I-GPE
FAST-O-
FOOD-O-
RESTAURANTS-O-
-------------------------------------------------------
4TH-O-
STREET-O-
PARKING-B-GPE
BERKELEY-B-GPE
CA-B-GPE
94710-O-
UNITED-B-GPE
STATES-I-GPE
OF-I-GPE
AMERICA-I-GPE
PARKING-O-
LOTS-O-
,-O-
METERS-O-
,-O-
GARAGES-O-
---------------------------------------

In [64]:
links = []
titles = []
snippets = []
i = 0

text1= '14 RBRT GAHANNA COLUMBUS OH 43230 RESTAURANTS'
text2= 'AAFES GUAM NEW MAIN STOR DALLAS TX 75265 UNITED STATES OF AMERICA MISC GENERAL MERCHANDISE'
text3= 'GAMESTOP.COM 19 2700 MIA DAYTON OH 45459 UNITED STATES OF AMERICA VIDEO AMUSEMENT GAME SUPPLY'
text4= 'SP  CHARGEDD HTTPSCHARGEDE MO 65201 UNITED STATES OF AMERICA ELECTRONICS STORES'
text5= 'UBER  CA  UNITED STATES OF AMERICA TAXICABS/LIMOUSINES'
text6= 'FRISCO LAKES GOLF CLUB WESTLAKEA TX 76262 UNITED STATES OF AMERICA PUBLIC GOLF COURSES'
text7 = 'HUA FU FAT BULL SEAFOO RIVERSIDE CA 92507 UNITED STATES OF AMERICA RESTAURANTS'
text8 = '860 LSU TEAM SHOP GREENWICH CT 6830 UNITED STATES OF AMERICA COSMETIC STORES'
textx = 'JARED GALLERIA 12430 FOOT  CA  UNITED STATES OF AMERICA JEWELRY STORES'
text9 = 'LOWES 02945  KY  UNITED STATES OF AMERICA HOME SUPPLY WAREHOUSE STORES'

search_query_list = [textx]
init_search_query = search_query_list[0]

search_query_proc_docs = list(textPreproc(nlp, search_query_list))


base_url = 'https://google.com/search?'
base_url_dym = 'https://google.com'
txt = {'q': init_search_query}
search_url = base_url + urllib.parse.urlencode(txt)
# search_url = 'http://httpbin.org/anything?json'
print(search_url)
params = {'url': search_url, 'custom_google':True}
request_result = send_request(params)

# parsing the available content
soup = BeautifulSoup(request_result.content, 'lxml')
did_you_mean = soup.find('a', {'class': 'gL9Hy'})
did_you_mean_link = ""
while did_you_mean is not None:
    did_you_mean_link = did_you_mean.get('href')
    did_you_mean_link = base_url_dym + did_you_mean_link
    print(did_you_mean_link)
    params = {'url': did_you_mean_link, 'custom_google':True}
    request_result = send_request(params)
    soup = BeautifulSoup(request_result.content, 'lxml')
    did_you_mean = soup.find('a', {'class': 'gL9Hy'})

https://google.com/search?q=LOWES+02945++KY++UNITED+STATES+OF+AMERICA+HOME+SUPPLY+WAREHOUSE+STORES
Response HTTP Status Code:  200


In [ ]:
print(soup.prettify())

In [65]:
local_listings = soup.find('div', {'class': 'UDZeY OTFaAf'})
if local_listings is not None:
    tree = etree.HTML(str(local_listings))
    format_tags = ['b', 'strong', 'i', 'em', 'mark', 'small', 'del', 'ins', 'sub', 'sup']
    # root = tree.getroot()

    text_list = []
    for element in tree.iter():
        text_to_append = ''
        if element.tag not in ['script', 'style', 'head', 'div'] and element.text is not None:
            text_to_append = element.text
            #print("In tag: " + element.tag, end="")
            #print("--- new text: " + text_to_append)
        
        if element.tag in format_tags and len(text_list) > 0:
            text_to_append = text_list[-1] + text_to_append
            text_list[-1] = text_to_append
            #print("In tag: " + element.tag, end="")
            #print("--- new text: " + text_to_append)
        elif text_to_append != '':
            text_list.append(text_to_append)

    print(text_list)
    text_list_lr = list(map(lambda x: x.lower(), text_list))
    if 'address' in text_list_lr and 'address' != text_list_lr[-1]:
        req_addr_ind = text_list_lr.index('address')
        req_addr = text_list[req_addr_ind + 1]
        print("\nExtracted address: ", end="")
        print(req_addr)
else:
    print("No local listings")

No local listings


In [44]:
local_listings_full = soup.find('div', {'class': 'ifM9O'})
local_listings = soup.find('div', {'class': 'UDZeY OTFaAf'})

local_pk = soup.find_all('div', {'class': 'cXedhc'})
if len(local_pk) != 0:
    for item in local_pk:
        name_elt = item.find('div', {'class': 'dbg0pd'})
        if name_elt is not None:
            name = name_elt.find('span').getText()
        addr_elt = item.find('div', {'class':'rllt__details'})
        if addr_elt is not None:
            addrs = addr_elt.find_all('div')
            addr_strs = []
            for addr in addrs:
                addr_strs.append(addr.getText())
        print("Name: " + name)
        for addr_str in addr_strs:
            print("Address: " + addr_str)
        print("--------------------------")
else:
    print("No local pack")


No local pack


In [41]:
search = soup.find_all('div', {'class': 'tF2Cxc'})
#rso > div:nth-child(9) > div > div > div.IsZvec > div
for h in search:
    links.append(h.a.get('href'))
    titles.append(h.a.find('h3', {'class':'LC20lb DKV0Md'}).getText())
    print(links[i])
    print(titles[i])
    spans1 = h.find('div', {'class':'IsZvec'})
    if spans1 is not None:
        spans2 = spans1.find('div', {'class': 'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'})
        if spans2 is not None:
            spans = spans2.find_all('span')
            if len(spans) > 0:
                snippets.append(spans[-1].getText())
            else:
                snippets.append("")
        else:
            snippets.append("")
    print(snippets[i])
    
    #spans = h.find('div.IsZvec').find('div.VwiC3b.yXK7lf.MUxGbd.yDYNvb.lyLwlc.lEBKkf').find_all('span')
    #snippets.append(spans[-1].getText())
    #print(links[i])
    #print(titles[i])
    #print(snippets[i])
    print("\n")
    i += 1
p = input("Done")

https://www.jared.com/
Jared | Your Local Jewelry Store for Engagement Rings ...
Jared is America's home for fine jewelry including diamond jewelry, gold jewelry, and engagement rings. We also carry a great selection of color gemstone ...


https://m.yelp.com/biz/jared-rancho-cucamonga
Jared - 12430 Foothill Blvd, Rancho Cucamonga, CA - Yelp
98 reviews of Jared "Every time we've been in here, the staff is really helpful. ... If you're in the market for jewelry, Jared is at least worth a look.


https://www.businessyab.com/explore/united_states/california/san_bernardino_county/rancho_cucamonga/east_foothill_boulevard/12430/jared.html
Jared, 12430 E Foothill Blvd, Rancho Cucamonga, CA 91739 ...
12430 E Foothill Blvd, Rancho Cucamonga, CA 91739, USA. Jared is a business providing services in the field of Jewelry store, Store, .


https://www.mapquest.com/us/california/jared-42288108
Jared 12430 Foothill Blvd Rancho Cucamonga, CA Jewelers
Get directions, reviews and information for Jared i

In [87]:
print(par_open + addr_pattern + par_close + or_pipe + par_open + biz_name_pattern + par_close)
string1 = 'he\lloth\ere'
print(string1)

((\d+[:]CARDINAL[;])*(\d+[:]FAC[;])*(\d+[:]PUNCT[;])*(\d+[:]LOC[;])*(\d+[:]PUNCT[;])*(\d+[:]ORDINAL[;])*(\d+[:]PUNCT[;])*((\d+[:]FAC[;])*(\d+[:]PUNCT[;])*)*(\d+[:]GPE[;])+(\d+[:]PUNCT[;])*((\d+[:]GPE[;])*(\d+[:]PUNCT[;])*)*(\d+[:]CARDINAL[;])*(\d+[:]PUNCT[;])*(\d+[:]GPE[;])*)|(((\d+[:]PRODUCT[;])|(\d+[:]WORK_OF_ART[;]))*(\d+[:]ORG[;])+)
he\lloth\ere


In [88]:
in_strings = [
    "AAFES KEESLER SHE,, MSCSS ,HATTIESBURG MS 39407 UNITED STATES OF AMERICA MISC GENERAL MERCHANDISE"
, "Bldg..... 2600 Camp-34567-Shelby,,, MS 39407----2003 United States"
, "AAFES KEESLER SHE,, MSCSS ,HATTIESBURG MS 39407 UNITED STATES OF AMERICA MISC GENERAL MERCHANDISE"
, "Building 2600 Camp Shelby, MS 39407-2003 United States"
, "GAMESTOP.COM 19 2700 MIA DAYTON OH 45459 UNITED STATES OF AMERICA VIDEO AMUSEMENT GAME SUPPLY"
]
out_snippets = textPreproc(nlp, snippets)
out_titles = textPreproc(nlp, titles)
orig_snippets = list(out_snippets)
orig_titles = list(out_titles)
# for i in range(len(orig_strings)):
#     print(orig_strings[i])
#     print("Type: " + str(type(orig_strings[i])))
# stringInfo(out_strings)
ent_snippets = entityString(orig_snippets)
ent_titles = entityString(orig_titles)
print(ent_snippets)
print(ent_titles)

['0:NOENT;1:NOENT;2:NOENT;3:NOENT;4:NOENT;5:NOENT;6:NOENT;7:ORG;8:NOENT;9:NOENT;10:NOENT;11:NOENT;12:NOENT;13:ORG;14:NOENT;15:PUNCT;16:NOENT;17:PUNCT;18:NOENT;19:NOENT;20:NOENT;21:PUNCT;22:NOENT;23:ORG;24:NOENT;25:NOENT;26:NOENT;27:NOENT;28:NOENT;', '0:NOENT;1:NOENT;2:PUNCT;3:NOENT;4:NOENT;5:GPE;6:GPE;7:PUNCT;8:GPE;9:PUNCT;10:ORG;11:ORG;12:ORG;13:NOENT;14:NOENT;15:NOENT;16:NOENT;17:NOENT;18:NOENT;19:NOENT;20:NOENT;21:NOENT;22:NOENT;23:GPE;24:NOENT;25:NOENT;26:NOENT;27:NOENT;28:NOENT;29:NOENT;', '0:FAC;1:FAC;2:FAC;3:PUNCT;4:NOENT;5:NOENT;6:NOENT;7:NOENT;8:NOENT;9:NOENT;10:NOENT;11:NOENT;12:ORG;13:ORG;14:PUNCT;15:NOENT;16:NOENT;17:DATE;18:DATE;19:DATE;20:DATE;21:PUNCT;22:NOENT;23:NOENT;24:NOENT;25:EVENT;26:EVENT;27:NOENT;', '0:ORG;1:ORG;2:ORG;3:ORG;4:ORG;5:NOENT;6:NOENT;7:NOENT;8:NOENT;9:ORG;10:NOENT;11:NOENT;12:NOENT;13:NOENT;14:PUNCT;15:NOENT;16:NOENT;17:ORG;18:ORG;19:NOENT;20:NOENT;21:PUNCT;22:ORG;23:NOENT;24:NOENT;25:PUNCT;26:ORG;27:NOENT;28:NOENT;29:NOENT;30:NOENT;31:NOENT;32:NOENT;

In [89]:
search_results_addr = extractPattern(addr_pattern, ent_snippets, ADDRESS)
search_results_biz = extractPattern(biz_name_pattern, ent_titles, BIZ_NAME)
matches_across_search_results_addr = getMatches(search_results_addr, orig_snippets)
matches_across_search_results_biz = getMatches(search_results_biz, orig_titles)
#match_list = [matches_across_search_results_addr, matches_across_search_results_biz]

In [91]:
search_query_proc = search_query_proc_docs[0].text
print(search_query_proc, end="\n\n")
#print(matches_across_search_results)
for (k, v) in matches_across_search_results_addr.items():
    print("Biz names found: ", end="")
    if len(matches_across_search_results_biz[k]) > 0:
        for match in matches_across_search_results_biz[k]:
            print(match, end=", ")
    else:
        print("None")    
    if len(v) > 0:
        for match in v:
            print(match)
            print("The Fuzz Score (partial ratio): " + str(fuzz.partial_ratio(search_query_proc, match)))
            print("The Fuzz Score (token set ratio): " + str(fuzz.token_set_ratio(search_query_proc, match)))
            print("The Fuzz Score (token sort ratio): " + str(fuzz.token_sort_ratio(search_query_proc, match)))
        print("Link: " + links[k])
    else:
        print("\nNo matches found in result " + str(k + 1))
    print("------------------------------------------------------\n")

860 LSU TEAM SHOP GREENWICH Connecticut 6830  COSMETIC STORES

Biz names found: LSU, LSU, LSU Tigers, 
No matches found in result 1
------------------------------------------------------

Biz names found: None

No matches found in result 2
------------------------------------------------------

Biz names found: LSU, 
No matches found in result 3
------------------------------------------------------

Biz names found: LSU, 
No matches found in result 4
------------------------------------------------------

Biz names found: Department of Justice, 138. Franconia. New Hampshire. 03580
The Fuzz Score (partial ratio): 22
The Fuzz Score (token set ratio): 39
The Fuzz Score (token sort ratio): 39
Link: https://www.doj.nh.gov/charitable-trusts/documents/registered-charities.pdf
------------------------------------------------------

Biz names found: The Cosmetics Company Store, 
No matches found in result 6
------------------------------------------------------

Biz names found: LSU Alumni Ass

In [38]:
pattern = "(a)*(b)+(a)*(c)+(a)*(b)*"
string_eg = "ttttttahahggfdgfgfaabbbbcccbbbbbbtfufyfu"
matches = re.findall(pattern, string_eg)
for match in matches:
    print(match)

('a', 'b', '', 'c', '', 'b')


In [39]:
doc = nlp("GAMESTOP.COM 19 2700 MIA DAYTON OH 4545 VIDEO AMUSEMENT GAME SUPPLY")
for token in doc:
    print(token.text + "-" + token.ent_iob_ + "-" + token.ent_type_)

GAMESTOP.COM-O-
19-O-
2700-O-
MIA-O-
DAYTON-O-
OH-B-GPE
4545-O-
VIDEO-O-
AMUSEMENT-O-
GAME-O-
SUPPLY-O-


In [40]:
string1 = "a b c"
string2 = "b c d"
string3 = "a b cd"
print(str(fuzz.ratio(string1, string3)))
print(str(fuzz.partial_ratio(string1, string3)))
print(str(fuzz.token_set_ratio(string1, string3)))
print(str(fuzz.token_sort_ratio(string1, string3)))

91
100
91
91


## Levenshtein distance function

## Example
For example, the Levenshtein distance between "kitten" and "sitting" is 3, since the following three edits change one into the other, and there is no way to do it with fewer than three edits:

- kitten → sitten (substitution of "s" for "k")
- sitten → sittin (substitution of "i" for "e")
- sittin → sitting (insertion of "g" at the end).

## Relationship with other edit distance metrics
There are other popular measures of edit distance, which are calculated using a different set of allowable edit operations. For instance,

- the Damerau–Levenshtein distance allows the transposition of two adjacent characters alongside insertion, deletion, substitution;
- the longest common subsequence (LCS) distance allows only insertion and deletion, not substitution;
- the Hamming distance allows only substitution, hence, it only applies to strings of the same length.
- the Jaro distance allows only transposition.

Edit distance is usually defined as a parameterizable metric calculated with a specific set of allowed edit operations, and each operation is assigned a cost (possibly infinite). This is further generalized by DNA sequence alignment algorithms such as the Smith–Waterman algorithm, which make an operation's cost depend on where it is applied.

## Upper and lower bounds
The Levenshtein distance has several simple upper and lower bounds. These include:

- It is at least the difference of the sizes of the two strings.
- It is at most the length of the longer string.
- It is zero if and only if the strings are equal.
- If the strings are the same size, the Hamming distance is an upper bound on the Levenshtein distance. The Hamming distance is the number of positions at which the corresponding symbols in the two strings are different.
- The Levenshtein distance between two strings is no greater than the sum of their Levenshtein distances from a third string (triangle inequality).

An example where the Levenshtein distance between two strings of the same length is strictly less than the Hamming distance is given by the pair "flaw" and "lawn". Here the Levenshtein distance equals 2 (delete "f" from the front; insert "n" at the end). The Hamming distance is 4.